In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv("hospital_train.csv", index_col=0)

In [42]:
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,,,,,,,,,,,,,,,,
161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0-10
159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,21-30
309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,11-20
279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,51-60
147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,51-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237869,12,a,9,Y,3,gynecology,R,B,3.0,82914,3.0,Emergency,Moderate,6,51-60,3966.0,More than 100 Days
254763,28,b,11,X,2,gynecology,R,F,2.0,40026,5.0,Urgent,Moderate,3,21-30,4005.0,51-60
69788,6,a,6,X,3,gynecology,Q,F,3.0,92346,2.0,Trauma,Minor,2,31-40,5215.0,31-40


In [3]:
class HospitalEncoder:

    @staticmethod
    def encode_admission(x):
        data = {'Urgent': 2, 'Trauma': 1, 'Emergency': 3}
        return data[x]

    @staticmethod
    def encode_severity(x):
        data = {'Moderate': 2, 'Extreme': 3, 'Minor': 1}
        return data[x]

    @staticmethod
    def encode_age(x):
        data = {'21-30': 2, '51-60': 5, '71-80': 7, '11-20': 1, 
        '31-40': 3, '0-10': 0, '61-70': 6, '41-50': 4, '81-90': 4, '91-100': 9}
        return data[x]

In [4]:
def normalize_df(df):
    for col in df.columns:
        df[col] = df[col]/df[col].max()
    return df

In [5]:
# Quitamos las columnas no relevantes
filtered = df.drop(['1', '4','9', '10', '11','16'], axis=1)

# Aplicamos los encoders
filtered['12'] = filtered['12'].apply(HospitalEncoder.encode_admission)
filtered['13'] = filtered['13'].apply(HospitalEncoder.encode_severity)
#filtered['15'] = filtered['15'].apply(HospitalEncoder.encode_age)

# Get dummies
features = filtered[['2', '3', '6', '7', '8', '15']]
features = pd.get_dummies(features)
filtered.drop(['2', '3', '6', '7', '8', '15'], axis=1, inplace=True)
train = pd.concat([filtered, features], axis=1)

# Creamos X y lo normalizamosC
X = np.array(normalize_df(train.drop(['17'], axis=1)))

# Cambiamos Nans de la columna 9 por la media
simp = SimpleImputer()
X = simp.fit_transform(X)

# Creamos y con encoder
le = LabelEncoder()
y = np.array(le.fit_transform(train['17']))

# y one-hot
#y = np.array(train['17'])
#y = keras.utils.to_categorical(y, num_classes=11)

# y sin modificar
#y = np.array(train['17'])

# Conjuntos train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [6]:
X_train.shape

(80000, 39)

In [34]:
y_train

array([2, 5, 2, ..., 1, 2, 2])

In [29]:
model = keras.Sequential([
    layers.InputLayer(input_shape=(39)),
    layers.Dense(64, activation='sigmoid'),
    layers.Dense(11, activation='sigmoid'),
    layers.Dense(1, activation='relu')
  ])

In [30]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
              loss="categorical_crossentropy", 
              metrics=['accuracy'])

In [31]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
2500/2500 [==============================] - 3s 928us/step - loss: 3.2333e-07 - accuracy: 0.2437
Epoch 2/5
2500/2500 [==============================] - 2s 970us/step - loss: 3.2333e-07 - accuracy: 0.2437
Epoch 3/5
2500/2500 [==============================] - 2s 910us/step - loss: 3.2333e-07 - accuracy: 0.2437
Epoch 4/5
2500/2500 [==============================] - 2s 905us/step - loss: 3.2333e-07 - accuracy: 0.2437
Epoch 5/5
2500/2500 [==============================] - 2s 874us/step - loss: 3.2333e-07 - accuracy: 0.2437


In [32]:
y_pred = model.predict(X_test)

In [33]:
pd.DataFrame(y_pred)

,0
0,0.607281
1,0.569077
2,0.574027
3,0.572042
4,0.569990
...,...
19995,0.567713
19996,0.534277
19997,0.595385
19998,0.565428


In [18]:
y_pred = le.inverse_transform(y_pred)
y_pred

C:\Users\jgnsa\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


ValueError: y contains previously unseen labels: [-0.20204103 -0.2020272  -0.20168462 ... -0.08061408 -0.08017939
 -0.07593304]